# GBDTsの学習・検証曲線とOptunaを使ったパラメーターチューニング

Kaggle内で学習曲線や検証曲線についての記載が少ないと思ったので、その算出について記載しました。
間違いがあったらご指摘いただけると嬉しいです。  

学習曲線・検証曲線については以下参照  
https://scikit-learn.org/stable/modules/learning_curve.html#validation-curve  
https://www.dataquest.io/blog/learning-curves-machine-learning/  

また、XGBoost等の勾配ブースティングモデルのパラメーターチューニングにおいて、最適パラメーターを決める便利なパッケージOptunaの使い方について記載しました。  


# ライブラリ―のインポート

In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb

import optuna
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.model_selection import validation_curve

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jan-2021/train.csv')
test  = pd.read_csv('../input/tabular-playground-series-jan-2021/test.csv')
sub = pd.read_csv('../input/tabular-playground-series-jan-2021/sample_submission.csv')

In [ ]:
train.shape

In [ ]:
train.describe()

In [ ]:
# Checking features and target columns
display(train.columns)
# Checking dtypes
display(train.info())

In [ ]:
features = ['cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7',
       'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13', 'cont14']

# 注意
本記事は、どちらかというと学習曲線や検証曲線の作成のために書いております。半分は自分の忘備録です。  
学習曲線や検証曲線は時間がかかるので全データ（30万）ではなく、5%ランダムサンプリングしております。もしフルで行いたい人はランダムサンプリングしないで行ってください。

In [ ]:
train_01 = train.sample(frac=0.05, replace=False, random_state=1)

In [ ]:
X = train_01[features]
y = train_01['target']

# XGBoost and Learning/Validation curves

## 学習曲線（Learning curve）
学習曲線とは、横軸にデータ数、縦軸にRSMEなどの指標を取り、TrainingセットとValidationセットとで別々にプロットしたものです。
過学習や未学習の検討における視覚的ツールとなります。
また、現モデルにおいて、これ以上サンプルを追加する価値があるかどうかをある程度検討できるので、データ収集継続に関して有用な示唆を提供できる可能性があります。
詳細は以下のUPL参照です。  
https://scikit-learn.org/stable/modules/learning_curve.html#validation-curve  
https://www.dataquest.io/blog/learning-curves-machine-learning/

In [ ]:
def learning_curves(estimator, title, X, y, cv= None, train_sizes=np.linspace(.3, 1.0, 5)):
    
    train_sizes, train_scores, validation_scores = \
        learning_curve(estimator, 
                       X,
                       y,
                       train_sizes = train_sizes,
                       cv = cv, 
                       scoring = 'neg_mean_squared_error')

    train_scores_mean = np.sqrt(-np.mean(train_scores, axis=1))
    train_scores_std = np.sqrt(np.std(train_scores, axis=1))/2
    validation_scores_mean = np.sqrt(-np.mean(validation_scores, axis=1))
    validation_scores_std = np.sqrt(np.std(validation_scores, axis=1))/2
    
    plt.figure(figsize=[6.5,4])
    plt.rcParams["font.size"] = 12
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label = 'Training error')
    plt.plot(train_sizes, validation_scores_mean, 'o-', color="g",label = 'Validation error')
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, validation_scores_mean - validation_scores_std,
                     validation_scores_mean + validation_scores_std, alpha=0.1,
                     color="g")
    
    plt.rcParams["font.size"] = 10
    plt.ylabel('RMSE', fontsize = 14)
    plt.xlabel('Training set size', fontsize = 14)
    title = title
    plt.title(title, fontsize = 18, y = 1.03)
    plt.legend()
    plt.ylim(0.575,0.775)

In [ ]:
params_xgb = {'lambda': 1,
 'alpha': 0,
 'colsample_bytree': 1,
 'subsample': 1,
 'learning_rate': 0.05,
 'max_depth': 6,
 'min_child_weight': 3,
 'random_state': 48}

In [ ]:
model_xgb = xgb.XGBRegressor(**params_xgb)

In [ ]:
title = 'Learning curve'
learning_curves(model_xgb, title, X, y, cv=5)

Trainingセットにおいては右肩上がりの、Validationセットにおいては若干右肩下がりのグラフになりました。典型的な学習曲線になります。  
このように、Trainingセットではサイズが小さいときには多くのデータにフィットするパラメーターが見つかるのでエラーが少なくなります。一方で、そのパラメーターはTrainingセットにのみ調整されているため、Validationセットにおいてはフィットが悪くエラーが大きくなります（図の左側）。  
サイズが大きくなるにつれて、フィットが平均的になっていくため、Trainingセットのフィットは調整され、一見悪くなる様に見えますが、一方でValidationセットではフィットが良くなっていきます。  
サイズを増やしても両者が離れたままだと過学習が残った状態といえます。
今回の図を見ると、徐々に両者が近づいているので、サンプルサイズをもっと多くすればよりフィットが良くなると考えられます。今回はデータの5%しか使っていませんので、全てのデータを使えばより良いフィットが期待できると言えそうです。

## 検証曲線（Validation curve）
検証曲線とは、横軸にパラメーター（例えば正則化パラメーターであるalpha）を取り、縦軸にRSMEなどの指標を取ります。TrainingセットとValidationセット毎にプロットしていくのは同様です。そのパラメーターを変化させたときにTrainingセットとValidationセットでどのような挙動を取るのかを視覚的に見る事が出来ます。最終的なパラメーターの決定根拠として利用されることがあります。  
注意：本来、検証曲線は利用可能なすべてのデータ（30万）で行うべきです。以下の結果は計算速度を優先したため、5%サンプリングのデータを使って検証曲線を作成しています。参考値として扱ってください。

後述するOptunaのような半自動設定の場合には、学習曲線や検証曲線を確認する事はないのかもしれませんね（なのでKaggle内では記述が少ないのかもと思いました）。

In [ ]:
def validation_curves(estimator, title, X, y,
                      cv= None, param_name= None, param_range=None):
    
    train_scores, test_scores = \
        validation_curve(estimator, 
                         X, 
                         y, 
                         param_name=param_name, 
                         param_range=param_range,
                         cv = cv,
                         scoring='neg_mean_squared_error', #'roc_auc'
                         n_jobs=4)
    train_scores_mean = np.sqrt(-np.mean(train_scores, axis=1))
    train_scores_std = np.sqrt(np.std(train_scores, axis=1))
    test_scores_mean = np.sqrt(-np.mean(test_scores, axis=1))
    test_scores_std = np.sqrt(np.std(test_scores, axis=1))

    plt.rcParams["font.size"] = 12
    plt.title(title, fontsize = 20)
    plt.xlabel(param_name, fontsize =14)
    plt.ylabel("Score", fontsize = 14)
    plt.ylim(0.5, 0.9)
    lw = 2
    plt.plot(param_range, train_scores_mean, label="Training score",
             color="darkorange", lw=lw)
    plt.fill_between(param_range, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.2,
                     color="darkorange", lw=lw)
    plt.plot(param_range, test_scores_mean, label="Cross-validation score",
             color="navy", lw=lw)
    plt.fill_between(param_range, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.2,
                     color="navy", lw=lw)
    plt.rcParams["font.size"] = 10
    plt.legend(loc="best")
    plt.show()

In [ ]:
param_range = np.linspace(0, 1, 10)
param_range

In [ ]:
param_name = "alpha"

In [ ]:
title = "Validation Curves for alpha"
validation_curves(model_xgb, title, X, y, cv=5, 
                  param_name = param_name, param_range = param_range)

alphaについては改善の余地がなさそうですね。

In [ ]:
param_name = "lambda"

In [ ]:
title = "Validation Curves for lambda"
validation_curves(model_xgb, title, X, y, cv=5, 
                  param_name = param_name, param_range = param_range)

lambdaも改善の余地がなさそうです。

In [ ]:
param_range = np.linspace(0.1, 1, 10)
param_range

In [ ]:
param_name = 'colsample_bytree'

In [ ]:
title = "Validation Curves for colsample"
validation_curves(model_xgb, title, X, y, cv=5, 
                  param_name = param_name, param_range = param_range)

colsample_bytreeもValidationセットにおいては変化があまりなさそうですね。

In [ ]:
param_name = 'subsample'

In [ ]:
title = "Validation Curves for subsample"
validation_curves(model_xgb, title, X, y, cv=5, 
                  param_name = param_name, param_range = param_range)

明確にどこが良いというのは無いように見えます。

In [ ]:
param_name = 'n_estimators'

In [ ]:
param_range = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]

In [ ]:
title = "Validation Curve for n_estimators"
validation_curves(model_xgb, title, X, y, cv=5, 
                  param_name = param_name, param_range = param_range)

## Optunaを利用したハイパーパラメーターチューニング
さて、上記の様に1つ1つみていくのも大事だと思いますが、あまり明確な結論は得られませんでした。また、パラメーターの数が多いのである程度自動的に決めてしまいたいと思います。  
ここではOptunaという便利な機能を利用したいと思います。
以下のNotebookが参考になります。  
https://www.kaggle.com/hamzaghanmi/xgboost-hyperparameter-tuning-using-optuna  


### 以下は時間がかかるのでGPU使用設定にしています。（それでも時間がかかる）
学習曲線の結果から、データ数が上昇すればするほど過学習が緩和され、予測性能が良くなる傾向がみられました。ここから行うOptunaを用いたチューニングでは、30万データ全てを用いる事にします。その結果を提出してスコアを見てみましょう。  
かなり時間がかかるのでGPUを使用する設定にしております。

In [ ]:
X = train[features]
y = train['target']

In [ ]:
def objective(trial,data=X,target=y):
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)
    param = {
        'tree_method':'gpu_hist',  # this parameter means using the GPU when training our model to speedup the training process
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 1),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 1),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.1, 0.2, 0.3,0.5,0.7,0.9]),
        'subsample': trial.suggest_categorical('subsample', [0.1, 0.2,0.3,0.4,0.5,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.0008, 0.01, 0.015, 0.02,0.03, 0.05,0.08,0.1]),
        'n_estimators': 4000,
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17,20,23,25]),
        'random_state': 48,
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 400),
    }
    model = xgb.XGBRegressor(**param)  
    
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=100,verbose=False)
    
    preds = model.predict(test_x)
    
    rmse = mean_squared_error(test_y, preds,squared=False)
    
    return rmse

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
study.best_trial.params

In [ ]:
Best_params_xgb = {'lambda': 0.001951466677835018,
 'alpha': 0.7843235982110978,
 'colsample_bytree': 0.5,
 'subsample': 0.8,
 'learning_rate': 0.01,
 'max_depth': 11,
 'min_child_weight': 205,
 'n_estimators': 3000,
 'random_state': 48,
 'tree_method':'gpu_hist'}

In [ ]:
X = train[features]
y = train['target']

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.15,random_state=42)
model_xgb = xgb.XGBRegressor(**Best_params_xgb)
model_xgb.fit(train_x,train_y,eval_set=[(test_x,test_y)],
              early_stopping_rounds=100,verbose=False)

feature importance

In [ ]:
importances = pd.Series(model_xgb.feature_importances_, index = features)
importances = importances.sort_values()
importances.plot(kind = "barh")
plt.title("imporance in the xgboost Model", fontsize=18)
plt.show()

In [ ]:
preds = model_xgb.predict(test_x)
rmse = mean_squared_error(test_y, preds,squared=False)
rmse

### 提出用ファイル作成


In [ ]:
test_X = test[features]

In [ ]:
preds = model_xgb.predict(test_X)

In [ ]:
sub['target']=preds
sub.to_csv('submission_xgb.csv', index=False)

# LightGBM and Learning/Validation curves
上記同様、学習曲線と検証曲線の作成をまず行います。  
次にOptunaを用いたパラメーターチューニングを行います。

## 学習曲線

In [ ]:
X = train_01[features]
y = train_01['target']

In [ ]:
params_lgb = {'num_leaves': 31,
 'min_data_in_leaf': 20,
 'min_child_weight': 0.001,
 'max_depth': -1,
 'learning_rate': 0.005,
 'bagging_fraction': 1,
 'feature_fraction': 1,
 'lambda_l1': 0,
 'lambda_l2': 0,
 'random_state': 48}

In [ ]:
model_lgb = lgb.LGBMRegressor(**params_lgb)

In [ ]:
title = 'Learning curve'
learning_curves(model_lgb, title, X, y, cv=5)

## 検証曲線
上記と同様にlambda_l1 (=alpha), lambda_l2, feature_fraction, bagging_fraction, n_estimatorsについて検証曲線を描きます。

In [ ]:
param_range = np.linspace(0, 1, 10)
param_range

In [ ]:
param_name = 'lambda_l1'

In [ ]:
title = "Validation Curves for lambda_l1"
validation_curves(model_lgb, title, X, y, cv=5, 
                  param_name = param_name, param_range = param_range)

In [ ]:
param_name = 'lambda_l2'

In [ ]:
title = "Validation Curves for lambda_l2"
validation_curves(model_lgb, title, X, y, cv=5, 
                  param_name = param_name, param_range = param_range)

In [ ]:
param_name = 'feature_fraction'

In [ ]:
title = "Validation Curves for feature_fraction"
validation_curves(model_lgb, title, X, y, cv=5, 
                  param_name = param_name, param_range = param_range)

In [ ]:
param_name = 'bagging_fraction'

In [ ]:
title = "Validation Curves for bagging_fraction"
validation_curves(model_lgb, title, X, y, cv=5, 
                  param_name = param_name, param_range = param_range)

In [ ]:
param_name = 'n_estimators'

In [ ]:
param_range = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]

In [ ]:
title = "Validation Curves for n_estimators"
validation_curves(model_lgb, title, X, y, cv=5, 
                  param_name = param_name, param_range = param_range)

## Optunaを用いたハイパーパラメーターチューニング
上記同様にOptunaにお任せします。

In [ ]:
X = train[features]
y = train['target']

In [ ]:
def objective_lgb(trial,data=X,target=y):
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)
    param = {
        'tree_method':'gpu_hist',  # this parameter means using the GPU when training our model to speedup the training process
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-3, 1),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-3, 1),
        'feature_framcion': trial.suggest_categorical('feature_framcion', [0.1, 0.2, 0.3,0.5,0.7,0.9]),
        'bagging_fraction': trial.suggest_categorical('bagging_framcion', [0.1, 0.2,0.3,0.4,0.5,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.0008, 0.01, 0.015, 0.02,0.03, 0.05,0.08,0.1]),
        'n_estimators': 4000,
        'num_leaves': trial.suggest_categorical('num_leaves', [31,50,150,200,250,300,350]),
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17,20,23,25]),
        'min_data_in_leaf': trial.suggest_categorical('min_data_in_leaf', [10,20,30]),
        'min_child_weight': trial.suggest_categorical('min_child_weight', [0.001,0.005, 0.01, 0.05, 0.1,0.5]),
        'random_state': 48
    }
    model = lgb.LGBMRegressor(**param)  
    
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=100,verbose=False)
    
    preds = model.predict(test_x)
    
    rmse = mean_squared_error(test_y, preds,squared=False)
    
    return rmse

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective_lgb, n_trials=50)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
study.best_trial.params

In [ ]:
Best_params_lgb = {'lambda_l2': 0.013616569506899653,
 'lambda_l1': 0.006495842188985166,
 'feature_framcion': 0.3,
 'bagging_framcion': 0.3,
 'learning_rate': 0.015,
 'num_leaves': 200,
 'max_depth': 25,
 'min_data_in_leaf': 30,
 'min_child_weight': 0.001,
 'n_estimators': 3000,
 'random_state': 48,
 'tree_method':'gpu_hist'}

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.15,random_state=42)
model_lgb = lgb.LGBMRegressor(**Best_params_lgb)
model_lgb.fit(train_x,train_y,eval_set=[(test_x,test_y)],
              early_stopping_rounds=100,verbose=False)

feature importance

In [ ]:
importances = pd.Series(model_lgb.feature_importances_, index = features)
importances = importances.sort_values()
importances.plot(kind = "barh")
plt.title("imporance in the lightGBM Model", fontsize=18)
plt.show()

In [ ]:
preds = model_lgb.predict(test_x)
rmse = mean_squared_error(test_y, preds,squared=False)
rmse

In [ ]:
test_X = test[features]

In [ ]:
preds = model_lgb.predict(test_X)

In [ ]:
sub['target']=preds
sub.to_csv('submission_lgb.csv', index=False)

### この記事がお役に立ったならば幸いです！
よろしければイイね　(・∀・)ｲｲﾈ!!　お願いいたします。